# BeautifullSoap

## Dependencies

In [ ]:
%pip install selenium_firefox
%pip install selenium

%pip install retry
%pip install timeout-decorator

!sudo apt-get -y install libglib2.0-dev

## Scraping Eudoxus Script

In [ ]:
# set options to be headless, ..
from selenium import webdriver


from selenium.webdriver.firefox.options import Options

options = Options()



# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from retry import retry
from timeout_decorator import timeout, TimeoutError



@retry(TimeoutError, tries=3)
@timeout(10)
def get_with_retry(wd, url):
    wd.get(url)

def get_tag_substring(assigned_substring, text, search_term):
  return text.split(': ')[1] if (search_term in text) else assigned_substring

def get_substring_after_first_occurance(source_string, split_on_string):
  return source_string.split(split_on_string, 1)[1]

def get_eudoxus_book_for_url(book_url):

  # create a webdriver instance, ready to use
  # wd = webdriver.Chrome('chromedriver',options=options)
  wd = webdriver.Firefox(options=options)
  
  try:

    get_with_retry(wd, book_url)

    
    # locator = (By.CSS_SELECTOR, '.gwt-Label .search-hyperlink')

    # book_code_element = WebDriverWait(wd, 30).until(
    #     EC.presence_of_element_located(locator)
    # )

    wait = WebDriverWait(wd, 10)
    
    book_code_locator_text = 'Κωδικός Βιβλίου στον Εύδοξο: '
    book_code_locator = (By.XPATH , f'//*[contains(text(),"{book_code_locator_text}")]')
    
    publication_date_locator_text = 'Έκδοση: '
    publication_date_locator = (By.XPATH , f'//*[contains(text(),"{publication_date_locator_text}")]')
    
    authors_locator_text = 'Συγγραφείς: '
    authors_locator = (By.XPATH , f'//*[contains(text(),"{authors_locator_text}")]')
    
    isbn_locator_text = 'ISBN: '
    isbn_locator = (By.XPATH , f'//*[contains(text(),"{isbn_locator_text}")]')
    
    publisher_locator_text = 'Διαθέτης (Εκδότης):'
    publisher_locator = (By.XPATH , f'//*[contains(text(),"{publisher_locator_text}")]')

    book_code_element = wait.until(EC.presence_of_element_located(book_code_locator))
    publication_date_element = wait.until(EC.presence_of_element_located(publication_date_locator))
    authors_element = wait.until(EC.presence_of_element_located(authors_locator))
    isbn_elements = wait.until(EC.presence_of_element_located(isbn_locator))
    publisher_elements = wait.until(EC.presence_of_element_located(publisher_locator))


    book_code= book_code_element.text.split(book_code_locator_text, 1)[1]
    publication_date = publication_date_element.text.split(publication_date_locator_text, 1)[1]
    authors = authors_element.text.split(authors_locator_text, 1)[1]
    isbn = isbn_elements.text.split(isbn_locator_text, 1)[1]
    publisher = publisher_elements.text.split(publisher_locator_text, 1)[1]
    

    # Split publication_date_info (1η/2010) into year and publication version number
    publication_year = publication_date.split('/')[1]
    publication_version = publication_date.split('/')[0]

    # Quit the WebDriver Instance
    wd.quit()

    return (book_code, publication_version, publication_year, authors, isbn, publisher)
  finally:
      wd.quit()


  
  


book_url = 'https://service.eudoxus.gr/search/#a/id:68373348/0'

book_details = get_eudoxus_book_for_url(book_url)

print(book_details)

In [68]:
!sudo playwright -y install-deps
%pip install --upgrade pip
%pip install playwright

!sudo apt-get install dialog apt-utils -y
!python -m playwright install-deps
!python -m playwright install

sudo: playwright: command not found
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  apt libapt-pkg6.0
Suggested packages:
  apt-doc aptitude | synaptic | wajig powermgmt-base
The following NEW packages will be installed:
  apt-utils dialog
The following packages will be upgraded:
  apt libapt-pkg6.0
2 upgraded, 2 newly installed, 0 to remove and 120 not upgraded.
Need to get 2,579 kB of archives.
After this operation, 2,118 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libapt-pkg6.0 amd64 2.0.6 [835 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 apt amd64 2.0.6 [1,296 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 apt-utils amd64 2.0.6 [216 kB]
Get:4 http

In [77]:
import asyncio
from playwright.async_api import async_playwright

async def main():
    async with async_playwright() as p:
        for browser_type in [p.chromium, p.firefox, p.webkit]:
            browser = await browser_type.launch()
            page = await browser.new_page()
            await page.goto('https://service.eudoxus.gr/search/#a/bookType:Published/0')
            # frame = page.querySelectorAll('search-resultsTable tbody > tr')
            book_tr = await page.query_selector_all(".search-resultsTable tbody > tr")
            # print(f'handle.inner_text(): {handle.inner_text()}')
            print(f'book_tr: {book_tr}')
            await browser.close()



try:
    loop = asyncio.get_running_loop()
except RuntimeError:  # 'RuntimeError: There is no current event loop...'
    loop = None

if loop and loop.is_running():
    print('Async event loop already running. Adding coroutine to the event loop.')
    tsk = loop.create_task(main())
    # ^-- https://docs.python.org/3/library/asyncio-task.html#task-object
    # Optionally, a callback function can be executed when the coroutine completes
    tsk.add_done_callback(
        lambda t: print(f'Task done with result={t.result()}  << return val of main()'))
else:
    print('Starting new event loop')
    asyncio.run(main())




Async event loop already running. Adding coroutine to the event loop.
book_tr: []
